This file contains the most promising models - time series based neural networks
There are a plethora included in the file with variations in the way they were trained. The best performing model is exported for use on the website

Author: Sean Brady
Created: Mar 10, 2024
Updated: officially Mar 24, 2024, but has been updated locally ever since as we have chosen to focus on these models

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def read_symbols(path): #func to read in the symbols from companies.txt
    with open(path,'r') as file: #open file
        return file.read().splitlines() #return array of company symbols

In [3]:
#get array of symbols
symbols_path = 'companies.txt'
symbols = read_symbols(symbols_path)
print(symbols)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'C

In [5]:
data = {} #dictionary to hold data
for symbol in symbols:
    data_path = f"company-data/{symbol}-data.csv" #path to company specific data
    try:
        df_ind = pd.read_csv(data_path)
        df_ind['target'] = df_ind['Close'].shift(-1) #create target variable (next days close variable)
        df_ind = df_ind.dropna() #drop NaN values
        print(f'{symbol}:\n')
        print(df_ind.shape)
        print('\n')
        data[symbol] = df_ind
    except:
        pass

MMM:

(5974, 26)


AOS:

(4682, 26)


ABT:

(5974, 26)


ABBV:

(2705, 26)


ACN:

(4712, 26)


ADBE:

(6004, 26)


AMD:

(6004, 26)


AES:

(6003, 26)


AFL:

(6002, 26)


A:

(6000, 26)


APD:

(6003, 26)


AKAM:

(4712, 26)


ALB:

(4712, 26)


ARE:

(4712, 26)


ALGN:

(4712, 26)


ALLE:

(2504, 26)


LNT:

(4712, 26)


ALL:

(6001, 26)


GOOGL:

(4842, 26)


GOOG:

(2427, 26)


MO:

(6004, 26)


AMZN:

(6001, 26)


AMCR:

(1116, 26)


AEE:

(6001, 26)


AAL:

(2500, 26)


AEP:

(6004, 26)


AXP:

(6004, 26)


AIG:

(6004, 26)


AMT:

(4712, 26)


AWK:

(3919, 26)


AMP:

(4572, 26)


AME:

(4712, 26)


AMGN:

(6002, 26)


APH:

(4712, 26)


ADI:

(6002, 26)


ANSS:

(4712, 26)


AON:

(4712, 26)


APA:

(6002, 26)


AAPL:

(6006, 26)


AMAT:

(6002, 26)


APTV:

(3015, 26)


ADM:

(6004, 26)


ANET:

(2378, 26)


AJG:

(4712, 26)


AIZ:

(4712, 26)


T:

(6006, 26)


ATO:

(4712, 26)


ADSK:

(6001, 26)


ADP:

(6002, 26)


AZO:

(6003, 26)


AVB:

(4712, 26)


AVY:

(6004, 26)




In [6]:
#arrays to hold the training and testing portions of each dataframe
train_frames = []
test_frames = []

for ticker, df in data.items(): #iterate over dictionary
    df['Ticker'] = ticker #add ticker as column
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index(['Ticker','Date']) #create multi-index
    #df = df.set_index(['Ticker', df.index]) #create multiindex [Ticker,Date]
    #df[ticker] = df #set as new dataframe
    
    #split at 80% mark
    split = int(len(df)*0.8)
    
    train_df = df.iloc[:split]
    test_df = df.iloc[split:]
    
    train_frames.append(train_df)
    test_frames.append(test_df)
    #train_frames.append(df.iloc[:split])
    #test_frames.append(df.iloc[split:])
    
train_df = pd.concat(train_frames)#, ignore_index=True)
test_df = pd.concat(test_frames)#, ignore_index=True)

print(train_df.shape)
print(test_df.shape)
print(train_df.head(10))
print(test_df.head(10))

(1792682, 25)
(448428, 25)
                      Open     High      Low    Close        Volume  \
Ticker Date                                                           
MMM    2000-03-29  31.1995  31.1995  30.6183  30.7074  4.594542e+06   
       2000-03-30  30.8634  32.1536  30.7074  31.1995  4.419489e+06   
       2000-03-31  31.6586  32.2485  31.0396  31.0396  3.443945e+06   
       2000-04-03  31.3332  32.3754  31.2828  32.1971  4.021207e+06   
       2000-04-04  32.2485  33.4970  31.3767  32.5846  6.779498e+06   
       2000-04-05  32.5555  33.2248  31.9134  31.9328  4.027360e+06   
       2000-04-06  31.9976  33.2461  31.9976  32.7125  2.459723e+06   
       2000-04-07  32.8045  33.3400  32.1119  32.1119  3.502765e+06   
       2000-04-10  32.1971  33.2461  32.1536  32.5110  2.955758e+06   
       2000-04-11  32.5110  34.0123  32.5110  33.4485  4.134358e+06   

                   daily_returns  2_day_ma   3_day_ma  5_day_ma   7_day_ma  \
Ticker Date                               

In [7]:
#make sure multi-indexing is working:
print(train_df.index.names)
print(test_df.index.names)

['Ticker', 'Date']
['Ticker', 'Date']


In [8]:
#print(data['MMM'].head(10))
print(list(data['MMM'].columns))

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'daily_returns', '2_day_ma', '3_day_ma', '5_day_ma', '7_day_ma', '10_day_ma', '20_day_ma', '30_day_ma', '2_day_volatility', '3_day_volatility', '5_day_volatility', '7_day_volatility', '10_day_volatility', '20_day_volatility', '30_day_volatility', '2_day_support', '2_day_resistance', '2_day_avg_volume', '2_day_stoch_k', 'target', 'Ticker']


In [9]:
#scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train = scaler.fit_transform(train_df.drop(['target'], axis=1))
scaled_test = scaler.transform(test_df.drop(['target'], axis=1))

In [10]:
def create_sequences(features, target, sequence_length):
    X = []
    y = []
    for i in range(len(features) - sequence_length):#loop through to create sequence data
        seq = features[i:i + sequence_length] #get a sequence of len sequence_length
        X.append(seq) #append to X
        target_value = target.iloc[i + sequence_length] #get target
        y.append(target_value)#append to y
    return np.array(X), np.array(y)


In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
def evaluate_model(predictions, y_true):
    mae = mean_absolute_error(y_true, predictions)
    rmse = np.sqrt(mean_squared_error(y_true, predictions))
    print(f"MAE: {mae}, RMSE: {rmse}")

In [12]:
#create train/test sequences
sequence_length = 7 # holds the length of each sequence
X_train, y_train = create_sequences(scaled_train, train_df['target'], sequence_length)
X_test, y_test = create_sequences(scaled_test, test_df['target'], sequence_length)

In [13]:
#define model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32,activation='relu'),
    Dense(32,activation='relu'),
    Dense(16,activation='relu'),
    Dense(8,activation='relu'),
    Flatten(),
    Dense(1)
])

#compile model
model.compile(optimizer=Adam(), loss='mse')

#train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

history = model.fit(X_train, y_train,
                    validation_split=0.1,
                    epochs=150,
                    batch_size=248,  # Start with a larger batch size
                    callbacks=[early_stopping, reduce_lr])

#model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# evaluate on test set
predictions = model.predict(X_test)
predictions = predictions.squeeze()
evaluate_model(predictions, y_test)#evaluate model accuracy


2024-03-28 17:46:42.909074: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/150
6835/6835 [==============================] - 28s 4ms/step - loss: 974.4443 - val_loss: 81.6247 - lr: 0.0010
Epoch 2/150
6835/6835 [==============================] - 27s 4ms/step - loss: 240.8208 - val_loss: 15.0579 - lr: 0.0010
Epoch 3/150
6835/6835 [==============================] - 26s 4ms/step - loss: 217.0476 - val_loss: 35.7736 - lr: 0.0010
Epoch 4/150
6835/6835 [==============================] - 31s 5ms/step - loss: 229.4323 - val_loss: 24.8036 - lr: 0.0010
Epoch 5/150
6835/6835 [==============================] - 27s 4ms/step - loss: 205.3668 - val_loss: 15.4186 - lr: 0.0010
Epoch 6/150
6835/6835 [==============================] - 26s 4ms/step - loss: 181.4888 - val_loss: 89.0368 - lr: 0.0010
Epoch 7/150
6835/6835 [==============================] - 27s 4ms/step - loss: 169.0916 - val_loss: 10.7442 - lr: 0.0010
Epoch 8/150
6835/6835 [==============================] - 26s 4ms/step - loss: 185.8880 - val_loss: 24.2162 - lr: 0.0010
Epoch 9/150
6835/6835 [=================

In [14]:
print(y_test.shape)
print(predictions.shape)
print(predictions[:10])

(471072,)
(471072,)
[185.39633 186.55916 189.28351 188.32915 189.25061 189.79044 190.79895
 190.754   191.58112 191.51753]


In [15]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(1883216, 7, 24)
(471072, 7, 24)
(1883216,)


In [20]:
print(list(X_test.columns))

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7, 128)            3200      
                                                                 
 dense_1 (Dense)             (None, 7, 64)             8256      
                                                                 
 dense_2 (Dense)             (None, 7, 64)             4160      
                                                                 
 dense_3 (Dense)             (None, 7, 32)             2080      
                                                                 
 dense_4 (Dense)             (None, 7, 32)             1056      
                                                                 
 dense_5 (Dense)             (None, 7, 16)             528       
                                                                 
 dense_6 (Dense)             (None, 7, 8)              1

In [17]:
#save the model
model_save_path = './exported_model'
tf.saved_model.save(model, model_save_path)


INFO:tensorflow:Assets written to: ./exported_model/assets


In [18]:
#save scaling fit
from joblib import dump

# Assume `scaler` is your fitted MinMaxScaler
dump(scaler, 'fitted_scaler.joblib')


['fitted_scaler.joblib']

In [31]:
#create train/test sequences
sequence_length = 30 # holds the length of each sequence
X_train, y_train = create_sequences(scaled_train, train_df['target'], sequence_length)
X_test, y_test = create_sequences(scaled_test, test_df['target'], sequence_length)

In [32]:
#define model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(128, activation='sigmoid'),
    Dense(64, activation='relu'),
    Dense(64, activation='sigmoid'),
    Dense(32,activation='relu'),
    Dense(32,activation='sigmoid'),
    Dense(16,activation='relu'),
    Dense(8,activation='sigmoid'),
    Flatten(),
    Dense(1)
])

#compile model
model.compile(optimizer=Adam(), loss='mse')

#train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

history = model.fit(X_train, y_train,
                    validation_split=0.1,
                    epochs=150,
                    batch_size=512,  # Start with a larger batch size
                    callbacks=[early_stopping, reduce_lr])

#model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# evaluate on test set
predictions = model.predict(X_test)
predictions = predictions.squeeze()
evaluate_model(predictions, y_test)#evaluate model accuracy


Epoch 1/150
3311/3311 [==============================] - 139s 42ms/step - loss: 23415.8340 - val_loss: 68.1347 - lr: 0.0010
Epoch 2/150
3311/3311 [==============================] - 126s 38ms/step - loss: 16087.1494 - val_loss: 96.7285 - lr: 0.0010
Epoch 3/150
3311/3311 [==============================] - 116s 35ms/step - loss: 12438.6797 - val_loss: 123.6766 - lr: 0.0010
Epoch 4/150
3311/3311 [==============================] - 115s 35ms/step - loss: 10134.6182 - val_loss: 62.3744 - lr: 0.0010
Epoch 5/150
3311/3311 [==============================] - 115s 35ms/step - loss: 8504.9453 - val_loss: 66.2030 - lr: 0.0010
Epoch 6/150
3311/3311 [==============================] - 116s 35ms/step - loss: 7254.1577 - val_loss: 61.7375 - lr: 0.0010
Epoch 7/150
3311/3311 [==============================] - 116s 35ms/step - loss: 6270.2437 - val_loss: 84.2939 - lr: 0.0010
Epoch 8/150
3311/3311 [==============================] - 114s 34ms/step - loss: 5454.9644 - val_loss: 84.2996 - lr: 0.0010
Epoch 9/150

In [33]:
#create train/test sequences
sequence_length = 15 # holds the length of each sequence
X_train, y_train = create_sequences(scaled_train, train_df['target'], sequence_length)
X_test, y_test = create_sequences(scaled_test, test_df['target'], sequence_length)

In [34]:
#define model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(128, activation='sigmoid'),
    Dense(64, activation='relu'),
    Dense(64, activation='sigmoid'),
    Dense(32,activation='relu'),
    Dense(32,activation='sigmoid'),
    Dense(16,activation='relu'),
    Dense(8,activation='sigmoid'),
    Flatten(),
    Dense(1)
])

#compile model
model.compile(optimizer=Adam(), loss='mse')

#train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

history = model.fit(X_train, y_train,
                    validation_split=0.1,
                    epochs=100,
                    batch_size=100,  # Start with a larger batch size
                    callbacks=[early_stopping, reduce_lr])

#model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# evaluate on test set
predictions = model.predict(X_test)
predictions = predictions.squeeze()
evaluate_model(predictions, y_test)#evaluate model accuracy


Epoch 1/100
16949/16949 [==============================] - 103s 6ms/step - loss: 19603.1055 - val_loss: 49.6242 - lr: 0.0010
Epoch 2/100
16949/16949 [==============================] - 97s 6ms/step - loss: 12284.1729 - val_loss: 38.6107 - lr: 0.0010
Epoch 3/100
16949/16949 [==============================] - 95s 6ms/step - loss: 9350.7949 - val_loss: 163.5926 - lr: 0.0010
Epoch 4/100
16949/16949 [==============================] - 94s 6ms/step - loss: 7525.7354 - val_loss: 86.0394 - lr: 0.0010
Epoch 5/100
16949/16949 [==============================] - 94s 6ms/step - loss: 6188.2803 - val_loss: 53.8467 - lr: 0.0010
Epoch 6/100
16949/16949 [==============================] - 102s 6ms/step - loss: 5155.9741 - val_loss: 42.8458 - lr: 0.0010
Epoch 7/100
16949/16949 [==============================] - 98s 6ms/step - loss: 4335.6284 - val_loss: 42.3247 - lr: 0.0010
Epoch 8/100
16949/16949 [==============================] - 95s 6ms/step - loss: 3671.3872 - val_loss: 44.9526 - lr: 0.0010
Epoch 9/100

In [11]:
#create train/test sequences
sequence_length = 45 # holds the length of each sequence
X_train, y_train = create_sequences(scaled_train, train_df['target'], sequence_length)
X_test, y_test = create_sequences(scaled_test, test_df['target'], sequence_length)

In [12]:
#define model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    #Dense(128, activation='sigmoid'),
    Dense(64, activation='relu'),
    #Dense(64, activation='sigmoid'),
    Dense(32,activation='relu'),
    #Dense(32,activation='sigmoid'),
    Dense(16,activation='relu'),
    #Dense(8,activation='sigmoid'),
    Flatten(),
    Dense(1)
])

#compile model
model.compile(optimizer=Adam(), loss='mse')

#train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

history = model.fit(X_train, y_train,
                    validation_split=0.1,
                    epochs=100,
                    batch_size=100,  # Start with a larger batch size
                    callbacks=[early_stopping, reduce_lr])

#model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# evaluate on test set
predictions = model.predict(X_test)
predictions = predictions.squeeze()
evaluate_model(predictions, y_test)#evaluate model accuracy


2024-03-24 12:31:46.022510: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
16949/16949 [==============================] - 113s 7ms/step - loss: 807.5663 - val_loss: 23.5057 - lr: 0.0010
Epoch 2/100
16949/16949 [==============================] - 100s 6ms/step - loss: 314.9342 - val_loss: 19.9003 - lr: 0.0010
Epoch 3/100
16949/16949 [==============================] - 102s 6ms/step - loss: 287.3748 - val_loss: 21.6230 - lr: 0.0010
Epoch 4/100
16949/16949 [==============================] - 96s 6ms/step - loss: 245.9908 - val_loss: 22.9199 - lr: 0.0010
Epoch 5/100
16949/16949 [==============================] - 96s 6ms/step - loss: 228.9581 - val_loss: 16.3764 - lr: 0.0010
Epoch 6/100
16949/16949 [==============================] - 101s 6ms/step - loss: 224.1772 - val_loss: 13.0825 - lr: 0.0010
Epoch 7/100
16949/16949 [==============================] - 92s 5ms/step - loss: 192.0335 - val_loss: 12.8621 - lr: 0.0010
Epoch 8/100
16949/16949 [==============================] - 90s 5ms/step - loss: 192.2803 - val_loss: 45.2967 - lr: 0.0010
Epoch 9/100
16949/16

NameError: name 'evaluate_model' is not defined

In [14]:
evaluate_model(predictions, y_test)#evaluate model accuracy

MAE: 3.6511066933893126, RMSE: 17.60244346683435


In [11]:
#create train/test sequences
sequence_length = 10 # holds the length of each sequence
X_train, y_train = create_sequences(scaled_train, train_df['target'], sequence_length)
X_test, y_test = create_sequences(scaled_test, test_df['target'], sequence_length)

In [13]:
#define model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(64, activation='relu'),
    Dense(32,activation='relu'),
    Dense(16,activation='relu'),
    Dense(8,activation='relu'),
    Dense(4,activation='relu'),
    Dense(2,activation='relu'),
    Flatten(),
    Dense(1)
])

#compile model
model.compile(optimizer=Adam(), loss='mse')

#train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

history = model.fit(X_train, y_train,
                    validation_split=0.1,
                    epochs=100,
                    batch_size=100,  # Start with a larger batch size
                    callbacks=[early_stopping, reduce_lr])

#model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# evaluate on test set
predictions = model.predict(X_test)
predictions = predictions.squeeze()
evaluate_model(predictions, y_test)#evaluate model accuracy


Epoch 1/100
16949/16949 [==============================] - 41s 2ms/step - loss: 30619.4512 - val_loss: 3467.3396 - lr: 0.0010
Epoch 2/100
16949/16949 [==============================] - 38s 2ms/step - loss: 29353.3555 - val_loss: 2726.4075 - lr: 0.0010
Epoch 3/100
16949/16949 [==============================] - 38s 2ms/step - loss: 28612.5645 - val_loss: 2440.5562 - lr: 0.0010
Epoch 4/100
16949/16949 [==============================] - 37s 2ms/step - loss: 28302.1719 - val_loss: 2420.7710 - lr: 0.0010
Epoch 5/100
16949/16949 [==============================] - 37s 2ms/step - loss: 28223.0293 - val_loss: 2454.5122 - lr: 0.0010
Epoch 6/100
16949/16949 [==============================] - 37s 2ms/step - loss: 28210.1973 - val_loss: 2473.2710 - lr: 0.0010
Epoch 7/100
16949/16949 [==============================] - 37s 2ms/step - loss: 28208.3965 - val_loss: 2481.6836 - lr: 0.0010
Epoch 8/100
16949/16949 [==============================] - 40s 2ms/step - loss: 28208.1836 - val_loss: 2485.2544 - lr:

In [14]:
#create train/test sequences
sequence_length = 10 # holds the length of each sequence
X_train, y_train = create_sequences(scaled_train, train_df['target'], sequence_length)
X_test, y_test = create_sequences(scaled_test, test_df['target'], sequence_length)

In [15]:
#define model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32,activation='relu'),
    Dense(16,activation='relu'),
    Dense(8,activation='relu'),
    Flatten(),
    Dense(1)
])

#compile model
model.compile(optimizer=Adam(), loss='mse')

#train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

history = model.fit(X_train, y_train,
                    validation_split=0.1,
                    epochs=100,
                    batch_size=100,  # Start with a larger batch size
                    callbacks=[early_stopping, reduce_lr])

#model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# evaluate on test set
predictions = model.predict(X_test)
predictions = predictions.squeeze()
evaluate_model(predictions, y_test)#evaluate model accuracy


Epoch 1/100
16949/16949 [==============================] - 75s 4ms/step - loss: 805.1049 - val_loss: 43.8564 - lr: 0.0010
Epoch 2/100
16949/16949 [==============================] - 78s 5ms/step - loss: 310.5793 - val_loss: 15.6786 - lr: 0.0010
Epoch 3/100
16949/16949 [==============================] - 73s 4ms/step - loss: 250.3736 - val_loss: 21.8742 - lr: 0.0010
Epoch 4/100
16949/16949 [==============================] - 73s 4ms/step - loss: 248.6731 - val_loss: 11.9359 - lr: 0.0010
Epoch 5/100
16949/16949 [==============================] - 76s 4ms/step - loss: 208.5689 - val_loss: 10.9322 - lr: 0.0010
Epoch 6/100
16949/16949 [==============================] - 78s 5ms/step - loss: 195.8464 - val_loss: 14.3706 - lr: 0.0010
Epoch 7/100
16949/16949 [==============================] - 79s 5ms/step - loss: 170.9386 - val_loss: 11.7647 - lr: 0.0010
Epoch 8/100
16949/16949 [==============================] - 76s 4ms/step - loss: 175.7600 - val_loss: 13.1456 - lr: 0.0010
Epoch 9/100
16949/16949 

In [12]:
#create train/test sequences
sequence_length = 45 # holds the length of each sequence
X_train, y_train = create_sequences(scaled_train, train_df['target'], sequence_length)
X_test, y_test = create_sequences(scaled_test, test_df['target'], sequence_length)

In [13]:
#define model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(128, activation='sigmoid'),
    Dense(64, activation='relu'),
    Dense(64, activation='sigmoid'),
    Dense(32,activation='relu'),
    Dense(32,activation='sigmoid'),
    Dense(16,activation='relu'),
    Dense(8,activation='sigmoid'),
    Flatten(),
    Dense(1)
])

#compile model
model.compile(optimizer=Adam(), loss='mse')

#train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

history = model.fit(X_train, y_train,
                    validation_split=0.1,
                    epochs=100,
                    batch_size=100,  # Start with a larger batch size
                    callbacks=[early_stopping, reduce_lr])

#model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# evaluate on test set
predictions = model.predict(X_test)
predictions = predictions.squeeze()
evaluate_model(predictions, y_test)#evaluate model accuracy


2024-04-21 23:11:14.731168: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
16134/16134 [==============================] - 213s 13ms/step - loss: 15139.2891 - val_loss: 130.3468 - lr: 0.0010
Epoch 2/100
16134/16134 [==============================] - 200s 12ms/step - loss: 7453.7202 - val_loss: 107.6562 - lr: 0.0010
Epoch 3/100
16134/16134 [==============================] - 197s 12ms/step - loss: 4807.6934 - val_loss: 130.1246 - lr: 0.0010
Epoch 4/100
16134/16134 [==============================] - 194s 12ms/step - loss: 3330.7817 - val_loss: 106.9492 - lr: 0.0010
Epoch 5/100
16134/16134 [==============================] - 196s 12ms/step - loss: 2315.7869 - val_loss: 272.7311 - lr: 0.0010
Epoch 6/100
16134/16134 [==============================] - 196s 12ms/step - loss: 1681.5916 - val_loss: 111.2111 - lr: 0.0010
Epoch 7/100
16134/16134 [==============================] - 198s 12ms/step - loss: 1347.6637 - val_loss: 109.1966 - lr: 0.0010
Epoch 8/100
16134/16134 [==============================] - 195s 12ms/step - loss: 1187.2383 - val_loss: 138.1558 - lr

In [14]:
#create train/test sequences
sequence_length = 21 # holds the length of each sequence
X_train, y_train = create_sequences(scaled_train, train_df['target'], sequence_length)
X_test, y_test = create_sequences(scaled_test, test_df['target'], sequence_length)

In [15]:
#define model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32,activation='relu'),
    Dense(32,activation='relu'),
    Dense(16,activation='relu'),
    Dense(8,activation='relu'),
    Flatten(),
    Dense(1)
])

#compile model
model.compile(optimizer=Adam(), loss='mse')

#train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

history = model.fit(X_train, y_train,
                    validation_split=0.1,
                    epochs=150,
                    batch_size=248,  # Start with a larger batch size
                    callbacks=[early_stopping, reduce_lr])

#model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# evaluate on test set
predictions = model.predict(X_test)
predictions = predictions.squeeze()
evaluate_model(predictions, y_test)#evaluate model accuracy


Epoch 1/150
6506/6506 [==============================] - 62s 9ms/step - loss: 1051.5784 - val_loss: 19.2871 - lr: 0.0010
Epoch 2/150
6506/6506 [==============================] - 57s 9ms/step - loss: 335.9425 - val_loss: 39.1898 - lr: 0.0010
Epoch 3/150
6506/6506 [==============================] - 54s 8ms/step - loss: 274.9128 - val_loss: 79.6180 - lr: 0.0010
Epoch 4/150
6506/6506 [==============================] - 51s 8ms/step - loss: 245.8609 - val_loss: 13.9030 - lr: 0.0010
Epoch 5/150
6506/6506 [==============================] - 51s 8ms/step - loss: 237.3326 - val_loss: 11.2595 - lr: 0.0010
Epoch 6/150
6506/6506 [==============================] - 51s 8ms/step - loss: 217.0293 - val_loss: 193.5535 - lr: 0.0010
Epoch 7/150
6506/6506 [==============================] - 52s 8ms/step - loss: 196.4398 - val_loss: 15.1233 - lr: 0.0010
Epoch 8/150
6506/6506 [==============================] - 53s 8ms/step - loss: 188.1384 - val_loss: 58.3274 - lr: 0.0010
Epoch 9/150
6506/6506 [===============

In [16]:
#create train/test sequences
sequence_length = 35 # holds the length of each sequence
X_train, y_train = create_sequences(scaled_train, train_df['target'], sequence_length)
X_test, y_test = create_sequences(scaled_test, test_df['target'], sequence_length)

In [17]:
#define model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    #Dense(128, activation='sigmoid'),
    Dense(64, activation='relu'),
    #Dense(64, activation='sigmoid'),
    Dense(32,activation='relu'),
    #Dense(32,activation='sigmoid'),
    Dense(16,activation='relu'),
    #Dense(8,activation='sigmoid'),
    Flatten(),
    Dense(1)
])

#compile model
model.compile(optimizer=Adam(), loss='mse')

#train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

history = model.fit(X_train, y_train,
                    validation_split=0.1,
                    epochs=100,
                    batch_size=100,  # Start with a larger batch size
                    callbacks=[early_stopping, reduce_lr])

#model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# evaluate on test set
predictions = model.predict(X_test)
predictions = predictions.squeeze()
evaluate_model(predictions, y_test)#evaluate model accuracy


Epoch 1/100
16134/16134 [==============================] - 91s 6ms/step - loss: 819.2036 - val_loss: 23.8225 - lr: 0.0010
Epoch 2/100
16134/16134 [==============================] - 75s 5ms/step - loss: 292.8583 - val_loss: 16.1900 - lr: 0.0010
Epoch 3/100
16134/16134 [==============================] - 71s 4ms/step - loss: 274.7835 - val_loss: 14.2757 - lr: 0.0010
Epoch 4/100
16134/16134 [==============================] - 71s 4ms/step - loss: 233.6570 - val_loss: 29.8696 - lr: 0.0010
Epoch 5/100
16134/16134 [==============================] - 71s 4ms/step - loss: 223.4670 - val_loss: 12.9636 - lr: 0.0010
Epoch 6/100
16134/16134 [==============================] - 72s 4ms/step - loss: 218.0679 - val_loss: 17.8999 - lr: 0.0010
Epoch 7/100
16134/16134 [==============================] - 78s 5ms/step - loss: 188.8941 - val_loss: 24.1921 - lr: 0.0010
Epoch 8/100
16134/16134 [==============================] - 72s 4ms/step - loss: 187.8905 - val_loss: 13.3859 - lr: 0.0010
Epoch 9/100
16134/16134 